In [129]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import joblib
import sys
import imodelsx.process_results
sys.path.append('../experiments/')
results_dir = '../results/'

r = imodelsx.process_results.get_results_df(results_dir)
experiment_filename = '../experiments/01_train_model.py'
r = imodelsx.process_results.fill_missing_args_with_default(r, experiment_filename)
# r.to_pickle('../results/agg.pkl')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 380/380 [00:00<00:00, 4913.92it/s]


In [130]:
def calc_mean_std_across_curves(shape_function_vals_list_list):
    '''
    shape_function_vals_list_list: list of lists of arrays
        num_seeds x num_features x num_points
    '''
    stds = []
    n_features = len(shape_function_vals_list_list[0])
    for feature_num in range(n_features):
        shape_function_vals = np.array(
            [
                shape_function_vals_list_list[i][feature_num]
                for i in range(len(shape_function_vals_list_list))
            ]
        )
        shape_function_mean = np.mean(shape_function_vals, axis=0)
        stds.append(np.mean(np.std(shape_function_vals, axis=0)))
    return np.mean(stds)

In [131]:
ravg = []
group_keys = [k for k in imodelsx.process_results.get_main_args_list(experiment_filename='01_train_model.py') if not k == 'seed']
rg = r.groupby(group_keys)
for group_key, group_idx in rg.groups.items():
    # print(group_idx)
    g = r.iloc[group_idx]
    # display(g)
    numeric_keys = [k for k in list(g.select_dtypes("number")) if not k in group_keys]
    row = g[numeric_keys].mean()
    for k in group_keys:
        row[k] = g[k].iloc[0]
    row['stability'] = calc_mean_std_across_curves(g['shape_function_vals_list'].values.tolist())
    ravg.append(row)
df = pd.DataFrame(ravg)

In [136]:
# df = df[df.dataset == "breast_cancer"]
df = df[~((df.n_boosting_rounds == 0) & (df.n_boosting_rounds_marginal == 0))]
print('ROC AUC:')
display(
    df.pivot_table(
        index="n_boosting_rounds",
        columns="n_boosting_rounds_marginal",
        values="roc_auc_test",
        aggfunc="mean",
    )
    .style.format(precision=3)
    .background_gradient(cmap="RdBu", axis=None)
)
print('Stability:')
display(
    df.pivot_table(
        index="n_boosting_rounds",
        columns="n_boosting_rounds_marginal",
        values="stability",
        aggfunc="mean",
    )
    .style.format(precision=4)
    .background_gradient(cmap="RdBu_r", axis=None)
)

ROC AUC:


n_boosting_rounds_marginal,0,5,25,125
n_boosting_rounds,,,,
0,nan,0.856,0.859,0.857
5,0.854,0.857,0.860,0.858
25,0.859,0.859,0.861,0.859
125,0.887,0.888,0.887,0.884


Stability:


n_boosting_rounds_marginal,0,5,25,125
n_boosting_rounds,,,,
0,nan,0.0024,0.0027,0.0030
5,0.0008,0.0029,0.0032,0.0033
25,0.0023,0.0042,0.0044,0.0043
125,0.0061,0.0068,0.0081,0.0077


Load an individual model

In [98]:
run_args = r.iloc[0]
model = joblib.load(join(run_args.save_dir_unique, 'model.pkl'))
model

TreeGAMClassifier(n_boosting_rounds=5, random_state=1)